# Regional Features

In [1]:
import numpy as np
import pandas as pd
import nibabel as nib
import itertools
import time

### Load data

In [2]:
df = pd.read_csv('../data/ATR_GT_Training.csv', header=None, names=['file','label'])
df.file = df.file.map(lambda x: x.replace("'",''))
df['img'] = df.file.map(lambda x: nib.load('../data/' + str(x) + '.nii.gz'))

### Define functions to calculate blocks and block means

In [3]:
def before_after(a, total_width):
    before = (total_width-a)//2
    after = total_width-before-a
    return (before, after)


def get_pad_width(x,y,z,total_width=400):
    return (before_after(x,total_width),
            before_after(y,total_width),
            before_after(z,total_width))


def get_padded_data(data, width=400):
    x,y,z = list(data.shape)
    return np.pad(data,get_pad_width(x,y,z),'constant',constant_values=0)


def get_block_means(data, width=400, num_blocks=4):
    block_means=[]
    ix = []
    for i in range(num_blocks):
        ix.append([width // num_blocks * i, width // num_blocks * (i+1)])
    count = 0
    for i, j, k in itertools.product(ix,ix,ix):
        block_means.append(np.mean(data[i[0]:i[-1],j[0]:j[-1],k[0]:k[-1]]))
    return block_means


def get_centered_block_means(data, width=400, num_blocks=10):
    ix = []
    for i in range(num_blocks):
        half_width = width//2
        half_region = width//num_blocks//2*(i+1)
        ix.append([half_width-half_region, half_width+half_region])
    centered_means = []
    for i in ix:
        centered_means.append(np.mean(padded[i[0]:i[-1],i[0]:i[-1],i[0]:i[-1]]))
    return centered_means

### Define feature labels

In [4]:
blocks = 4
block_labels = ['block_' + str(x) + '_mean' for x in range(1,blocks**3+1)]
centered_blocks = 10
centered_block_labels = ['centered_block_' + str(x) + '_mean' for x in range(1,centered_blocks+1)]
new_features = block_labels + centered_block_labels
df = df.reindex(columns = df.columns.tolist() + new_features)

### Calculate 64 block means

In [5]:
features_regional = []
for i, img in enumerate(df.img):
    padded = get_padded_data(img.get_fdata())
    region_means = get_block_means(padded)
    centered_means = get_centered_block_means(padded)
    features = region_means + centered_means
    features_regional.append(features)
    if i % 25 == 0 and i != 0:
        print('Finished ' + str(i) + ' images.')
print('Finished.')
np.save('features_regional.npy', np.array(features_regional))

Finished 25 images.
Finished 50 images.
Finished 75 images.
Finished 100 images.
Finished 125 images.
Finished 150 images.
Finished 175 images.
Finished 200 images.
Finished 225 images.
Finished 250 images.
Finished 275 images.
Finished 300 images.
Finished 325 images.
Finished 350 images.
Finished 375 images.
Finished 400 images.
Finished 425 images.
Finished 450 images.
Finished 475 images.
Finished 500 images.
Finished 525 images.
Finished 550 images.
Finished 575 images.
Finished 600 images.
Finished 625 images.
Finished 650 images.
Finished 675 images.
Finished 700 images.
Finished 725 images.
Finished 750 images.
Finished 775 images.
Finished 800 images.
Finished 825 images.
Finished 850 images.
Finished 875 images.
Finished 900 images.
Finished 925 images.
Finished 950 images.
Finished 975 images.
Finished 1000 images.
Finished 1025 images.
Finished 1050 images.
Finished 1075 images.
Finished 1100 images.
Finished 1125 images.
Finished 1150 images.
Finished 1175 images.
Finished